In [7]:
import ipywidgets as widgets

In [8]:
import matplotlib.pyplot as plt

In [9]:
import numpy as np
import pandas as pd
import edhec_risk_kit as erk
def show_cppi(n_scenarios=50,mu=0.07,sigma=0.15,m=3,floor=0.,riskfree_rate=0.03,y_max=100):
    """
    Plot the results of a monte carlo simulation of CPPI
    """
    start =100
    sim_rets = erk.gbm(n_scenarios=n_scenarios,mu=mu,sigma=sigma,prices=False,steps_per_year=12)
    risky_r=pd.DataFrame(sim_rets)
    #run the back-test
    btr=erk.run_cppi(risky_r=pd.DataFrame(risky_r),riskfree_rate=riskfree_rate,m=m,start=start,floor=floor)
    wealth=btr["Wealth"]
    y_max=wealth.values.max()*y_max/100
    terminal_wealth =wealth.iloc[-1]
    tw_mean =terminal_wealth.mean()
    tw_median =terminal_wealth.median()
    failure_mask=np.less(terminal_wealth,start*floor)
    n_failures=failure_mask.sum()
    p_fail= n_failures/n_scenarios
    tv=terminal_wealth
    e_shortfall =np.dot(tv-start*floor, failure_mask/n_failures) if n_failures >0 else 0.0
    #Plot!
    fig, (wealth_ax,hist_ax) =plt.subplots(nrows=1,ncols=2,sharey=True,gridspec_kw={'width_ratios':[3,2]},figsize=(24,9))
    plt.subplots_adjust(wspace=0.0)

    wealth.plot(ax=wealth_ax, legend=False,alpha=0.3,color="indianred")
    wealth_ax.axhline(y=start,ls=":",color="black")
    wealth_ax.axhline(y=start*floor,ls="--",color="red")
    wealth_ax.set_ylim(top=y_max)
    
    terminal_wealth.plot.hist(ax=hist_ax,bins=50,ec='w',fc='indianred',orientation='horizontal')
    hist_ax.axhline(y=start,ls=":",color="black")
    hist_ax.axhline(y=tw_mean,ls=":",color="blue")
    hist_ax.axhline(y=tw_median,ls=":",color="purple")
    hist_ax.annotate(f"Mean: ${int(tw_mean)}", xy=(0.7,0.9),xycoords='axes fraction',fontsize=24)
    hist_ax.annotate(f"Median: ${int(tw_median)}", xy=(0.7,0.85),xycoords='axes fraction',fontsize=24)
    if (floor> 0.01):
       hist_ax.axhline(y=start*floor,ls="--",color="red",linewidth=3)
       hist_ax.annotate(f"Violations:{n_failures}({p_fail*100:2.2f}%)\nE(shortfall)=${e_shortfall:2.2f}",xy=(0.5,0.5),xycoords='axes fraction',fontsize=24)
       hist_ax.annotate(f"mean of terminal wealth:{tv.mean()}",xy=(0.4,0.4),xycoords='axes fraction',fontsize=24)

        
cppi_controls = widgets.interactive(show_cppi,
                                    n_scenarios=widgets.IntSlider(min=1,max=1000,                                       step=5,value=50),
                                    mu=(0.,+2,.01),
                                    sigma=(0,0.50,0.05),
                                    floor=(0,2,0.01),
                                    m=(1,20,0.5),
                                    riskfree_rate=(0,0.05,0.01),
                                    y_max=widgets.IntSlider(min=0,max=100,step=1,                                      value=100,description="Zoom Y Axis")

)
display(cppi_controls) 

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…

In [10]:
def run_cppi_quiz(risky_r,safe_r=None,m=3,start=1000,floor=0.8,riskfree_rate=0.03,drawdown=None,rebals_per_year=None):
    """
    Run a backtest of the CPPI strategy,given a set of returns for the risky asset.
    Returns a dictionary containing: Asset Value History, Risk Budget History, risky weight history.
    """
    #Set the CPPI parameters.
    dates=risky_r.index
    n_steps=len(dates)
    account_value=start
    floor_value=start*floor
    peak=start
    if isinstance(risky_r,pd.Series):
        risky_r=pd.DataFrame(risky_r,columns=["R"])
        #Here, even if we are given a series, to make it work, we initialize risky returns as data frame.
    if safe_r is None:
        safe_r=pd.DataFrame().reindex_like(risky_r)
        safe_r.values[:]=riskfree_rate/12# fast way to set all values to a number.

    #Set up some data frames to save all intermediate values.
    account_history=pd.DataFrame().reindex_like(risky_r)
    cushion_history=pd.DataFrame().reindex_like(risky_r)
    risky_w_history=pd.DataFrame().reindex_like(risky_r)

    for step in range(1,n_steps+1):
        if drawdown is not None: #doing this to make floor value dynamic.
            peak=np.maximum(peak,account_value)
            floor_value=peak*(1-drawdown)
        cushion=(account_value-floor_value)/account_value
        if rebals_per_year is not None:
           p=int(12/rebals_per_year)
           if step%p==0:
              cushion=(start-floor_value)/start
              risky_w=m*cushion
              safe_w=1-risky_w
        risky_w =m*cushion
        risky_w = np.minimum(risky_w,1)
        risky_w = np.maximum(risky_w,0)
        safe_w=1- risky_w
        risky_alloc=account_value*risky_w
        safe_alloc=account_value*safe_w
        #Update the account value for this time step.
        account_value=risky_alloc*(1+risky_r.iloc[step-1])+safe_alloc*(1+safe_r.iloc[step-1])
        # Save the values for later reference.
        cushion_history.iloc[step-1] = cushion
        risky_w_history.iloc[step-1] = risky_w
        account_history.iloc[step-1] = account_value
       

    risky_wealth=start*(1+risky_r).cumprod()        
    
    backtest_result = {
        "Wealth": account_history,
        "Risky Wealth": risky_wealth,
        "Risk Budget": cushion_history,
        "Risky Allocation": risky_w_history,
        "multiplier m":m,
        "start": start,
        "floor": floor,
        "risky_r": risky_r,
        "safe_r": safe_r
    }


    return backtest_result
    

In [12]:
import numpy as np
def show_cppi(n_scenarios=50,mu=0.07,sigma=0.15,m=3,floor=0.,riskfree_rate=0.03,y_max=100,rebals_per_year=2):
    """
    Plot the results of a monte carlo simulation of CPPI
    """
    start =100
    sim_rets = erk.gbm(n_scenarios=n_scenarios,mu=mu,sigma=sigma,prices=False,steps_per_year=12)
    risky_r=pd.DataFrame(sim_rets)
    #run the back-test
    btr=run_cppi_quiz(risky_r=pd.DataFrame(risky_r),riskfree_rate=riskfree_rate,m=m,start=start,floor=floor,rebals_per_year=rebals_per_year)
    wealth=btr["Wealth"]
    y_max=wealth.values.max()*y_max/100
    terminal_wealth =wealth.iloc[-1]
    tw_mean =terminal_wealth.mean()
    tw_median =terminal_wealth.median()
    failure_mask=np.less(terminal_wealth,start*floor)
    n_failures=failure_mask.sum()
    p_fail= n_failures/n_scenarios
    tv=terminal_wealth
    e_shortfall =np.dot(tv-start*floor, failure_mask/n_failures) if n_failures >0 else 0.0
    #Plot!
    fig, (wealth_ax,hist_ax) =plt.subplots(nrows=1,ncols=2,sharey=True,gridspec_kw={'width_ratios':[3,2]},figsize=(24,9))
    plt.subplots_adjust(wspace=0.0)

    wealth.plot(ax=wealth_ax, legend=False,alpha=0.3,color="indianred")
    wealth_ax.axhline(y=start,ls=":",color="black")
    wealth_ax.axhline(y=start*floor,ls="--",color="red")
    wealth_ax.set_ylim(top=y_max)
    
    terminal_wealth.plot.hist(ax=hist_ax,bins=50,ec='w',fc='indianred',orientation='horizontal')
    hist_ax.axhline(y=start,ls=":",color="black")
    hist_ax.axhline(y=tw_mean,ls=":",color="blue")
    hist_ax.axhline(y=tw_median,ls=":",color="purple")
    hist_ax.annotate(f"Mean: ${int(tw_mean)}", xy=(0.7,0.9),xycoords='axes fraction',fontsize=24)
    hist_ax.annotate(f"Median: ${int(tw_median)}", xy=(0.7,0.85),xycoords='axes fraction',fontsize=24)
    if (floor> 0.01):
       hist_ax.axhline(y=start*floor,ls="--",color="red",linewidth=3)
       hist_ax.annotate(f"Violations:{n_failures}({p_fail*100:2.2f}%)\nE(shortfall)=${e_shortfall:2.2f}",xy=(0.5,0.5),xycoords='axes fraction',fontsize=24)
       hist_ax.annotate(f"mean of terminal wealth:{tv.mean()}",xy=(0.4,0.4),xycoords='axes fraction',fontsize=24)
       hist_ax.annotate(f"diff:{tv.max()-tv.min()}",xy=(0.2,0.8),xycoords='axes fraction',fontsize=24)
       hist_ax.annotate(f"worst case:{tv.min()}",xy=(0.4,0.6),xycoords='axes fraction',fontsize=24)

        
cppi_controls = widgets.interactive(show_cppi,
                                    n_scenarios=widgets.IntSlider(min=1,max=1000,                                       step=5,value=50),
                                    mu=(0.1,+2,.01),
                                    sigma=(0.1,0.50,0.05),
                                    floor=(0,2,0.01),
                                    m=(1,20,0.5),
                                    riskfree_rate=(-1,0.05,0.01),
                                    rebals_per_year=(0,12,1),
                                    y_max=widgets.IntSlider(min=0,max=100,step=1,                                      value=100,description="Zoom Y Axis")

)
display(cppi_controls) 

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…